In [1]:
import numpy as np
import h5py
import eagle_python as eagle
import read_eagle as read_particle
from multiprocessing import Pool
import multiprocessing
from tqdm import trange, tqdm


ModuleNotFoundError: No module named 'eagle_python'

In [4]:
basePath= '/Simulations/Eagle/RefL0100N1504/'
snapnum=28
fname   = eagle.snapshot.snapPath(basePath,snapnum)


/Simulations/Eagle/RefL0100N1504//snapshot_028_z000p000/snap_028_z000p000.0.hdf5


In [2]:
import numpy as np
import h5py
#read data
r200=np.zeros([1])
mass=np.zeros([1])
pos=np.zeros([1,3])
GN=np.zeros([1])
Vel=np.zeros([1,3])
starmass=np.zeros([1])
FirstID=np.zeros([1])
starspin=np.zeros([1,3])
for i in np.arange(256):
    f = h5py.File('/Simulations/Eagle/RefL0100N1504/groups_028_z000p000/eagle_subfind_tab_028_z000p000.'+np.str(i)+'.hdf5', 'r')
    r200=np.concatenate((r200,f['FOF']['Group_R_Crit200']))

    pos=np.concatenate((pos,f['FOF']['GroupCentreOfPotential']))
    mass=np.concatenate((mass,f['FOF']['Group_M_Crit200']))
    GroupNumber=f["Subhalo"]["GroupNumber"]
    GN=np.concatenate((GN,GroupNumber))

    Vel=np.concatenate((Vel,f["Subhalo"]["Velocity"])) 
    starmass=np.concatenate((starmass,f['Subhalo']['Stars']['Mass']))    
    FirstID=np.concatenate((FirstID,f['FOF']['FirstSubhaloID']))
    starspin=np.concatenate((starspin,f['Subhalo']['Stars']['Spin']))    
    f.close()
r200 = np.delete(r200, 0, 0)
Vel = np.delete(Vel, 0, 0)
pos = np.delete(pos,0,0)
mass = np.delete(mass,0,0)
GN = np.delete(GN,0,0)
starmass=np.delete(starmass,0,0)
FirstID=np.delete(FirstID,0,0)
starspin=np.delete(starspin,0,0)

Vel=Vel[np.int32(FirstID)]
starmass=starmass[np.int32(FirstID)]
starspin=starspin[np.int32(FirstID)]

num=(np.where(mass>=1))
r200=r200[num]
pos=pos[num]
mass=mass[num]
Vel=Vel[num]
starmass=starmass[num]
starspin=starspin[num]


print("finish 1")


finish 1


In [3]:
select=(np.where(starmass>=1e-1))#stellar mass>10^9 M_sun
r200=r200[select]
pos=pos[select]
mass=mass[select]
Vel=Vel[select]
starmass=starmass[select]
starspin=starspin[select]

In [6]:
len(starspin)

6220

In [5]:
np.save("/home/heyi/spin-cosmic-web-alignment/starspin.npy",starspin)

In [5]:
print("len(mass)=%s\n"%(len(mass)))

len(mass)=6220



In [19]:
basePath= '/Simulations/Eagle/RefL0100N1504/'
snapnum=28


def parts_in_region(basePath,snapnum, partType, centre: 'cMpc/h', region_length:'cMpc/h', fields):
    #print('\n##\n#region_length is box\'s length of selected region, not radius',)
    #fname   = basePath+"/snapshot_0%d_z000p000/snap_0%d_z000p000.0.hdf5" % (snapnum, snapnum)
    fname   = eagle.snapshot.snapPath(basePath,snapnum)
    itype = partType
    result = {}
    # make sure fields is not a single element
    if isinstance(fields, str):
        fields = [fields]
    
    # Open the snapshot
    snap = read_particle.EagleSnapshot(fname)
    # Specify the region to read (coords. are in comoving Mpc/h)
    xmin = centre[0]- 0.5*region_length
    xmax = centre[0]+ 0.5*region_length
    ymin = centre[1]- 0.5*region_length
    ymax = centre[1]+ 0.5*region_length
    zmin = centre[2]- 0.5*region_length
    zmax = centre[2]+ 0.5*region_length
    snap.select_region(xmin, xmax, ymin, ymax, zmin, zmax)

    #print ("# Number of particles in this region = %d" % snap.count_particles(itype))
    # Read positions and IDs of particles of type itype in the specified region.
    for field in fields:
        result[field]= snap.read_dataset(itype, field)
    snap.close()
    return result
print("finish 2")

parttype=1
cores = multiprocessing.cpu_count()

def Jcompute(i):
    halop=parts_in_region(basePath,snapnum,parttype,pos[i],r200[i],['Coordinates','Velocity'])        
    halopr=halop['Coordinates']-pos[i]
    halopv=halop['Velocity']-Vel[i]    
    r3d=np.sqrt(np.sum(halopr*halopr, axis = 1))
    halopr=halopr[np.where(r3d < r200[i]),:][0]
    halopv =halopv[np.where(r3d < r200[i]),:][0]
    Jall=np.cross(halopr,halopv)
    J=Jall.sum(axis=0)
    
    return J

finish 2


In [21]:
if __name__ == '__main__':
    cores = multiprocessing.cpu_count()
    with Pool(cores) as p:
        a=np.arange(len(mass))
        J=p.map(Jcompute, a)

In [ ]:
len(J)

In [23]:
J

[array([1.18494491e+08, 3.95555251e+08, 2.40642857e+08]),
 array([-1.55281018e+09, -1.44130525e+09,  2.12108679e+08]),
 array([-2.97153632e+08,  8.24285187e+07,  5.00323504e+08]),
 array([-1.74035790e+08,  4.69741895e+08, -4.26623466e+07]),
 array([-4.57550122e+08,  4.12172627e+07, -4.41004500e+07]),
 array([-1.11971895e+09,  1.40892168e+08, -8.01171320e+07]),
 array([-2.05792198e+07,  4.75987739e+08, -3.33755948e+08]),
 array([-49407207.83855559,   4611095.30346892,   5611045.97405727]),
 array([-43594350.93841112, -49718105.01755495,  35553684.22315245]),
 array([ 5.5345299e+07,  1.5174078e+08, -1.0500098e+08]),
 array([82423365.29182923, 34973631.61420093, 55854731.46163828]),
 array([-1.81010345e+08, -1.73411585e+08,  2.12157897e+08]),
 array([-12775016.36934678, -44392902.11261027, -52369055.85455114]),
 array([-26705249.47006413,  10895280.56822018, -26194248.35285496]),
 array([-38746558.24798902,  99638153.11563085, -38779704.00621793]),
 array([ 65087535.04246143, -68291955.54

In [22]:
Galneedtype=[
    ( 'Pos'                  , (np.float32, 3)), 
    ( 'Spin'            , (np.float32, 3)),
    ( 'Mass'           , np.float32),
]
halo=np.zeros(len(mass),dtype=Galneedtype)
halo['Pos']=pos
halo["Spin"]=J
halo['Mass']=mass

np.save("halo_pos_J_mass_test_200921_circle.npy",halo)

In [10]:
type(np.where(np.sum(pos*pos, axis = 1)>1))

tuple

In [ ]:
halopr=halopr[np.where(r3d < r200[i]),:][0]
halopv =halopv[np.where(r3d < r200[i]),:][0]